In [21]:
import dotenv
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from typing import List

import pandas as pd

dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("GPT_KEY")

# https://python.langchain.com/docs/expression_language/get_started/


MODEL = ChatOpenAI(model="gpt-4")


In [22]:
roteiro_id = 2
user_response = "Em ciência da computação, Hashmap é uma estrutura de dados especial que associa a chave de pesquisa a valores. Seu objetivo é a partir de uma chave simples, fazer uma busca rápida e obter o valor desejado. É algumas vezes traduzidas traduzida como tabela de dispersão."

In [23]:
# df_messages é o dataframe com as mensagens referentes a conversa atual  
df_messages = pd.read_csv("src/services/data/conversation.csv", sep=";", encoding="UTF-8")

# df_script é o dataframe com os roteiros
df_script = pd.read_csv("src/services/data/roteiros.csv", sep=";", encoding="UTF-8")

# df_script_stages é o dataframe com as etapas dos roteiros
df_script_stages = pd.read_csv("src/services/data/roteiro_stages.csv", sep=";", encoding="UTF-8")

conversa_id = df_messages['CONVERSATION_ID'].max() + 1

# pegamos o contexto, ou seja, todas as mensagens trocadas até agora na conversa
context = df_messages[df_messages['CONVERSATION_ID'] == conversa_id]

# se o contexto é vazio, pegamos a primeira mensagem do roteiro
if len(context) == 0:
    # Salva a mensagem na base de dados
    first_message = df_script_stages[(df_script_stages['ROTEIRO_ID'] == roteiro_id) & (df_script_stages['STAGE'] == 0)]
    first_message = {
        "CONVERSATION_ID" : [conversa_id], 
        "ROTEIRO_ID" : [roteiro_id], 
        "STAGE" : [0], 
        "MENSAGEM_ID" : [1], 
        "TRANSCRIPT" : [first_message['OPTION'].values[0]], 
        "SENDER": ["CHAT"]
    }
    df_messages = pd.concat([df_messages, pd.DataFrame(first_message)], ignore_index=True)
    df_messages.to_csv('src/services/data/conversation.csv', index=False, sep=";")
    context = df_messages[df_messages['CONVERSATION_ID'] == conversa_id]

In [24]:
df_script_stages

,ROTEIRO_ID,STAGE,OPTION
0,1,0,"Doutor, boa tarde. Eu sou a Sandra, e vim rece..."
1,1,1,Eu sou contra relacionamentos abertos pois ele...
2,1,1,Eu não tenho nenhum tipo de contato com pessoa...
3,1,1,Eu consigo assobiar e chupar cana ao mesmo tempo
4,2,0,"Vamos começar a entrevista? Primeiramente, com..."
5,2,1,Isso não faz sentido algum! Essa estrutura de ...
6,2,1,"Sua resposta foi mais ou menos, mas não inclui..."
7,2,1,"Perfeito, você acertou! A estrutura de dados H..."


In [25]:
context

,CONVERSATION_ID,ROTEIRO_ID,STAGE,MENSAGEM_ID,TRANSCRIPT,SENDER


In [26]:
df_messages

,CONVERSATION_ID,ROTEIRO_ID,STAGE,MENSAGEM_ID,TRANSCRIPT,SENDER
0,NaN,2,0,1,"Vamos começar a entrevista? Primeiramente, com...",CHAT


In [27]:
df_script = df_script[df_script['ROTEIRO_ID'] == roteiro_id]

In [28]:
df_script

,ROTEIRO_ID,CONTEXT,CHAT,USER
1,2,Você deve participar de uma entrevista de prog...,Entrevistador,Entrevistado


In [29]:
def buildPrompt(context: pd.DataFrame, df_scripts: pd.Series, message:str, chat_role:str, options_qtty:int):

    prompt = "O contexto da Atividade é:\n" + df_scripts["CONTEXT"] + "\n" + f"Você deve atuar como o {chat_role}" + "\n\n Essa é a conversa até agora: \n"

    messages = context['TRANSCRIPT'].to_list()
    senders = context['SENDER'].to_list()


    for i in range(len(context)):
        prompt += df_scripts[senders[i]] + " - " + messages[i] + "\n\n"

    prompt += "A última mensagem do/a " + df_scripts["USER"] + " foi:\n" + message + "\n"

    prompt+= "Qual seria a próxima resposta mais apropriada, entre as seguintes:\n"

    for i in range(options_qtty):
        prompt += "{Resposta " + f"{i+1}" + "}\n"

    prompt += "Responda somente com o número correspondente a resposta escolhida"

    return prompt

In [30]:
stage = context['STAGE'].max()+1
m_id = context['MENSAGEM_ID'].max()

In [31]:
stage

nan

In [32]:
m_id

nan

In [33]:
df_script_stages = df_script_stages[(df_script_stages['ROTEIRO_ID'] == roteiro_id) & (df_script_stages['STAGE'] == stage)]
options = df_script_stages['OPTION'].astype(str).to_list()

In [34]:
options

[]

In [35]:
chat_role = df_script["CHAT"].values[0]
options_qtty = len(options)
prompt = buildPrompt(context=context, df_scripts=df_script.iloc[0], message=user_response, chat_role=chat_role, options_qtty=options_qtty)

In [36]:
prompt

'O contexto da Atividade é:\nVocê deve participar de uma entrevista de programação\nVocê deve atuar como o Entrevistador\n\n Essa é a conversa até agora: \nA última mensagem do/a Entrevistado foi:\nEm ciência da computação, Hashmap é uma estrutura de dados especial que associa a chave de pesquisa a valores. Seu objetivo é a partir de uma chave simples, fazer uma busca rápida e obter o valor desejado. É algumas vezes traduzidas traduzida como tabela de dispersão.\nQual seria a próxima resposta mais apropriada, entre as seguintes:\nResponda somente com o número correspondente a resposta escolhida'

In [37]:
{f"Resposta{i+1}":options[i] for i in range(len(options))}

{}

In [38]:
def getResponse(options: List[str], prpt: str) -> int:

    prompt = ChatPromptTemplate.from_template(prpt)
    output_parser = StrOutputParser()

    chain = prompt | MODEL | output_parser

    n = chain.invoke({f"Resposta {i+1}":options[i] for i in range(len(options))})

    return int(n)

In [39]:
response = getResponse(options=options, prpt=prompt)

ValueError: invalid literal for int() with base 10: '1. Certo, você pode me explicar a diferença entre um Hashmap e um Hashtable?\n2. Entendi. Agora, você pode me contar sobre sua experiência com a linguagem Python?\n3. Você poderia me dar um exemplo d

In [ ]:
response

3